In [55]:
import pandas as pd
from gensim.parsing.preprocessing import *
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

df_corpus = pd.read_csv('corpus.csv',index_col=0)

corpus = []
for topic in df_corpus:
    corpus += df_corpus[topic].tolist()

answ_corpus = pd.Series(corpus)
answ_corpus = answ_corpus.apply(strip_tags)
answ_corpus = answ_corpus.apply(strip_multiple_whitespaces)
corpus = pd.Series(corpus)
corpus = corpus.apply(lambda x: x.lower())
corpus = corpus.apply(strip_tags)
corpus = corpus.apply(strip_punctuation2)
corpus = corpus.apply(strip_numeric)
corpus = corpus.apply(lambda x: re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', u' ', x))
corpus = corpus.apply(lambda x: re.sub('«|»', ' ', x))
corpus = corpus.apply(strip_multiple_whitespaces)
corpus = corpus.apply(lambda x: x.partition(' – ')[2])



import stop_words
from nltk.stem.snowball import RussianStemmer

corpus_tokens = []
inds_to_drop = []

for i, sentence in enumerate(corpus[:]):
    tmp_tokens = []
    sp = sentence.split()
    for word in sp:
        if word not in stop_words.get_stop_words('ru'):
            if morph.word_is_known(word):
                tmp_tokens.append(word)
    if len(tmp_tokens) > 0:
        corpus_tokens.append(tmp_tokens)
    else:
        inds_to_drop.append(i)
#     break
        
print(len(corpus_tokens))#, len(texts))



stemmer = RussianStemmer()

corpus_tokens_stem = []

for i, tokens in enumerate((corpus_tokens[:])):
    tmp = [stemmer.stem(word) for word in tokens]
    corpus_tokens_stem.append(tmp)
#     break
    
print(len(corpus_tokens_stem))


clear_corpus = []
for token_list in corpus_tokens_stem:
    if len(' '.join(token_list))<2:
        print(token_list)
    clear_corpus.append(' '.join(token_list))
clear_corpus[0]


vectorizer = TfidfVectorizer()
vectorizer.fit(clear_corpus)
X = vectorizer.transform(clear_corpus)


598
598


# Gensim LDA

In [59]:
dictionary = corpora.Dictionary(corpus_tokens_stem)
X_bow = [dictionary.doc2bow(text) for text in corpus_tokens_stem]
lda_gensim = gensim.models.ldamodel.LdaModel(X_bow, num_topics=N_TOPICS, id2word=dictionary, iterations=10)
lda_gensim.print_topics()

[(0,
  '0.019*"мурманск" + 0.006*"област" + 0.004*"росс" + 0.003*"дел" + 0.003*"сообща" + 0.003*"летн" + 0.003*"нов" + 0.003*"мест" + 0.003*"рубл" + 0.003*"работ"'),
 (1,
  '0.012*"мурманск" + 0.007*"автомобил" + 0.005*"област" + 0.004*"рубл" + 0.004*"работ" + 0.004*"росс" + 0.004*"дел" + 0.004*"водител" + 0.003*"жител" + 0.003*"рф"'),
 (2,
  '0.012*"мурманск" + 0.006*"област" + 0.004*"проект" + 0.004*"работ" + 0.004*"водител" + 0.003*"автомобил" + 0.003*"программ" + 0.003*"декабр" + 0.003*"нов" + 0.003*"дел"'),
 (3,
  '0.012*"мурманск" + 0.005*"област" + 0.004*"росс" + 0.004*"автомобил" + 0.004*"январ" + 0.004*"рубл" + 0.003*"работ" + 0.003*"город" + 0.003*"рф" + 0.003*"дел"'),
 (4,
  '0.012*"мурманск" + 0.005*"област" + 0.005*"полиц" + 0.004*"жител" + 0.004*"нов" + 0.004*"работ" + 0.003*"рубл" + 0.003*"участник" + 0.003*"свобод" + 0.003*"автомобил"'),
 (5,
  '0.013*"мурманск" + 0.005*"водител" + 0.004*"автомобил" + 0.004*"област" + 0.004*"работ" + 0.004*"дел" + 0.004*"летн" + 0.004*"

In [4]:
from sklearn.decomposition import LatentDirichletAllocation

In [60]:
N_TOPICS = 6
max_iteration = 500
batch_size = 10
evaluate_every=1
verb=1
learning_method='batch'

In [61]:
lda = LatentDirichletAllocation(n_components=N_TOPICS, max_iter=max_iteration, batch_size=batch_size, evaluate_every=evaluate_every, verbose=verb, 
                                        n_jobs=4, learning_method=learning_method)
# .fit(X)

In [62]:
lda.fit(X)

iteration: 1 of max_iter: 500, perplexity: 60109.6052
iteration: 2 of max_iter: 500, perplexity: 56885.9946
iteration: 3 of max_iter: 500, perplexity: 52872.0977
iteration: 4 of max_iter: 500, perplexity: 48138.7124
iteration: 5 of max_iter: 500, perplexity: 42960.6708
iteration: 6 of max_iter: 500, perplexity: 38576.6075
iteration: 7 of max_iter: 500, perplexity: 34922.3327
iteration: 8 of max_iter: 500, perplexity: 32293.9316
iteration: 9 of max_iter: 500, perplexity: 30156.9182
iteration: 10 of max_iter: 500, perplexity: 28659.6656
iteration: 11 of max_iter: 500, perplexity: 27582.2955
iteration: 12 of max_iter: 500, perplexity: 26780.9283
iteration: 13 of max_iter: 500, perplexity: 26132.5747
iteration: 14 of max_iter: 500, perplexity: 25546.8534
iteration: 15 of max_iter: 500, perplexity: 25127.0340
iteration: 16 of max_iter: 500, perplexity: 24814.8644
iteration: 17 of max_iter: 500, perplexity: 24642.9543
iteration: 18 of max_iter: 500, perplexity: 24489.9499
iteration: 19 of ma

LatentDirichletAllocation(batch_size=10, doc_topic_prior=None,
             evaluate_every=1, learning_decay=0.7, learning_method='batch',
             learning_offset=10.0, max_doc_update_iter=100, max_iter=500,
             mean_change_tol=0.001, n_components=6, n_jobs=4,
             n_topics=None, perp_tol=0.1, random_state=None,
             topic_word_prior=None, total_samples=1000000.0, verbose=1)

In [63]:
from sklearn.pipeline import make_pipeline
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import Normalizer
svd = TruncatedSVD(n_components=N_TOPICS, n_iter=100).fit(X)

lsa = make_pipeline(svd, Normalizer(copy=False))

In [64]:
keys = list(vectorizer.vocabulary_.keys())
values = list(vectorizer.vocabulary_.values())

_ind = np.argsort(values)
words_sorted = np.asarray(keys)[_ind]

print(len(keys), len(values), len(words_sorted))

7109 7109 7109


In [65]:
df_corpus.columns

Index(['auto', 'crime', 'culture', 'education'], dtype='object')

In [72]:
import numpy as np
stub = [0] * N_TOPICS
threshold = 0.1

for i in range(N_TOPICS):
    
    szd = pd.Series(np.sort(svd.components_[i])[-10:])
    szd_mask = szd.apply(lambda x: False if x<threshold else True)
    
    print(szd[szd_mask].values)
    
    ind = np.argsort(lda.components_[i])[-szd[szd_mask].shape[0]:]
    print(words_sorted[ind])
    
    print()
#     break

[0.11974542 0.12172687 0.12689177 0.13078238 0.1311683  0.14097139
 0.15137524 0.17164023 0.17526135 0.22917545]
['композитор' 'географ' 'находк' 'свидан' 'сочинен' 'скакалк' 'итальянск'
 'беллин' 'столов' 'диктант']

[0.11542823 0.11580785 0.11583691 0.1189751  0.13569249 0.14079865
 0.14775426 0.18384743 0.25467627 0.28934121]
['жител' 'срок' 'уголовн' 'дел' 'рубл' 'свобод' 'лишен' 'област' 'полиц'
 'мурманск']

[0.10358589 0.1099553  0.11000508 0.11068345 0.11152459 0.1199679
 0.1369217  0.15086606 0.15183271 0.1840259 ]
['пленк' 'форт' 'зенитн' 'ракетн' 'топлив' 'кремлевск' 'литр' 'дизельн'
 'уаз' 'копеек']

[0.1060818  0.12212066 0.15457948 0.18857641 0.19580236]
['незнаком' 'краб' 'куб' 'вход' 'свободн']

[0.10332364 0.11979795 0.12549766 0.15889184 0.16770367 0.21908653
 0.22107959 0.22728758 0.23558164]
['кладбищ' 'поход' 'воздержа' 'вблиз' 'владимир' 'зон' 'солов' 'взрывн'
 'радиус']

[0.10151888 0.11014743 0.11992617 0.14006268 0.15721703 0.1602395
 0.16370343 0.22057178 0.37